### 基于FastText的意图分类

在这里我们训练一个FastText意图识别模型，并把训练好的模型存放在模型文件里。 意图识别实际上是文本分类任务，需要标注的数据：每一个句子需要对应的标签如闲聊型的，任务型的。但在这个项目中，我们并没有任何标注的数据，而且并不需要搭建闲聊机器人。所以这里搭建的FastText模型只是一个dummy模型，没有什么任何的作用。这个模块只是为了项目的完整性，也让大家明白FastText如何去使用，仅此而已。

In [1]:
import fasttext
import pandas as pd
from tqdm import tqdm
import numpy as np
import pickle

In [3]:
# 读取数据：导入在preprocessor.ipynb中生成的data/question_answer_pares.pkl文件，并将其保存在变量QApares中
with open('data/question_answer_pares.pkl','rb') as f:
    QApares = pickle.load(f)

In [4]:
# 这一文档的目的是为了用fasttext来进行意图识别，将问题分为任务型和闲聊型两类
# 训练数据集是任务型还是闲聊型本应由人工打上标签，但这并不是我们的重点。我们的重点是教会大家如何用fasttext来进行意图识别
# 所以这里我们为数据集随机打上0或1的标签，而不去管实际情况如何

#fasttext的输入格式为：单词1 单词2 单词3 ... 单词n __label__标签号
#我们将问题数据集整理为fasttext需要的输入格式并为其随机打上标签并将结果保存在data/fasttext/fasttext_train.txt和data/fasttext/fasttext_test.txt中
with open('data/fasttext/fasttext_train.txt','w') as f:
    for content in QApares[:int(0.7*len(QApares))].dropna().question_after_preprocessing:
        f.write('%s __label__%d\n' % (' '.join(content), np.random.randint(0,2)))
with open('data/fasttext/fasttext_test.txt','w') as f:
    for content in QApares[int(0.7*len(QApares)):].dropna().question_after_preprocessing:
        f.write('%s __label__%d\n' % (' '.join(content), np.random.randint(0,2)))

In [5]:
#使用fasttext进行意图识别，并将模型保存在classifier中
classifier = fasttext.train_supervised('data/fasttext/fasttext_train.txt',     #训练数据文件路径
                                       label="__label__",      #类别前缀
                                       dim=100,       #向量维度
                                       epoch=5,       #训练轮次
                                       lr=0.1,        #学习率
                                       wordNgrams=2,      #n-gram个数
                                       loss='softmax',    #损失函数类型
                                       thread=5,          #线程个数, 每个线程处理输入数据的一段, 0号线程负责loss输出
                                       verbose=True)

In [6]:
#使用训练好的fasttext模型进行预测
classifier.predict('今天 月亮 真 圆 啊')

(('__label__0',), array([0.50145501]))

In [7]:
#使用训练好的fasttext模型对测试集文件进行评估
classifier.test('data/fasttext/fasttext_test.txt')

(30000, 0.5028, 0.5028)

In [8]:
#保存模型
classifier.save_model('model/fasttext.ftz')